In [1]:
import pandas as pd
import numpy as np
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from tqdm import tqdm_notebook as tqdm
import re

from multiprocessing import Pool

ModuleNotFoundError: No module named 'sparse_dot_topn'

In [2]:
df = pd.read_csv("export_rws_jultooct18_v3.csv")

In [3]:
len(np.unique(df['data.user_email'].str.lower()).tolist())

69616

# Idea 1: TFIDF with N grams

Advantages: 1. Scalable, fast, can be distributed using Spark

Disadvantages: How accurate the results are? Are there a lot of false positives?

In [4]:
def ngrams(string, n=3):
    """
     
    Get the n-grams of the strings with some cleaning such as removing punctuations like ,_./ or \sBD.
    """
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [5]:
#The code below generate tf-idf for each email in email_names
from sklearn.feature_extraction.text import TfidfVectorizer

email_names = np.unique(df['data.user_email'].str.lower()).tolist()[:10000]
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(email_names)

In [6]:
tf_idf_matrix.shape

(10000, 14938)

In [7]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    """
    Given two csr sparse matrices, we calculate the cosine similarity between the two matrices.
    In our case, we calculate the cosine similarity of the tfidf representation of each email addresses in our list.
    We are using sparse_dot_topn algorithm in performing sparse matrix multiplication followed by topn multiplication
    result selection. This algorithm is customized cython function of comparing large tfidf feature vectors and picking
    up the best matches using cosine similarity measure.
    """
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [8]:
%%time

#We are getting the top 100 matches of each email with a similarity measure greater than or equal to 0.5.
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 100, 0.5)

CPU times: user 965 ms, sys: 6.53 ms, total: 971 ms
Wall time: 1 s


In [9]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    """
    Unpack the resulting sparse matrix. We set an option to look into the first n values of the matches matrix.
    
    """
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similarity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similarity})

In [10]:
matches_df = get_matches_df(matches, user_emails, top=11468)
matches_df = matches_df[(matches_df['similarity'] > 0.5) & (matches_df['similarity'] <= 0.99999)] # Remove all exact matches
matches_df.sort_values(by=['similarity'], ascending=0)

,left_side,right_side,similarity
1754,afiqahahmadmazlan@gmail.com,afiqahahmadmazlan@gmail.co,0.999201
1750,afiqahahmadmazlan@gmail.co,afiqahahmadmazlan@gmail.com,0.999201
3044,alicecranberry23@gmail.com,alicecranberry23@gmail.con,0.976109
3046,alicecranberry23@gmail.con,alicecranberry23@gmail.com,0.976109
6702,askoddsynow@gmail.com,askoddsynow@gmail.con,0.975656
6704,askoddsynow@gmail.con,askoddsynow@gmail.com,0.975656
2669,albertlam0912@gmail.com,albertlam0912@gmail.con,0.974450
2671,albertlam0912@gmail.con,albertlam0912@gmail.com,0.974450
4503,andresin.zsanett@gmail.con,andresin.zsanett@gmail.com,0.972773
4501,andresin.zsanett@gmail.com,andresin.zsanett@gmail.con,0.972773


In [11]:
matches_df[matches_df['left_side'] == 'carenlou0810@gmail.com']

,left_side,right_side,similarity
10436,carenlou0810@gmail.com,carenlou@gmail.com,0.52716


# Idea 2: Use Voldemort FuzzyWuzzy Idea

In [12]:
import re
from collections import Counter
from fuzzywuzzy import fuzz, process

/Users/algerremirata/miniconda3/envs/data_integration/lib/python3.4/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
def match_score(a, b):
    confUnRaw = fuzz.ratio(a[0], b[0])
    confUn = fuzz.ratio(a[1], b[1])
    conf = fuzz.ratio(a[3], b[3])
    return (confUnRaw + confUn * 2 + conf) / 4


def match_process(s):
    s = s.split("@", 1)
    if len(s) < 2:
        s.insert(1, "")
    cleaned_username = re.sub(r"[\d\W]", "", s[0])
    s.insert(1, cleaned_username)
    clean_full = "@".join(s[1:3])
    s.append(clean_full)
    # s.append(orig)
    # 0 raw username, 1 clean username, 2 domain (blank if doesn't exist), 3 clean full#, 4 raw full
    return s

In [14]:
def similar_email_vold(user_email, user_emails):
    user_emails = Counter(user_emails)
    user_email_list = [i for i in user_emails]
    matches = process.extractBests(user_email, user_email_list, processor=match_process, scorer=match_score, score_cutoff=85, limit=20)
    scores = []
    sum_score = 0
    for match, confidence in matches:
        numTx = user_emails[match]
        if match != user_email:
            scores.append(numTx * 10)
    if scores:
        sum_score = min([sum(scores), 50])
        # if isEmoto(transaction):
        # 	this.addReason("\n<br>Extra emoto score", sum_score)
    return sum_score

Using a unique or not in the list of emails to worked on.

In [15]:
%%time

user_emails = df['data.user_email'].tolist()[:1000]
user_emails = [email.lower() for email in list(set(user_emails))]

df4 = pd.DataFrame({'emails': list(user_emails)})
df4['score'] = 0
for idx, address in enumerate(tqdm(user_emails)):
    score = similar_email_vold(address, user_emails)
    df4.loc[idx, 'score'] = score


CPU times: user 4h 36min 54s, sys: 39.6 s, total: 4h 37min 34s
Wall time: 7h 11min 22s


# Optimizing Voldemort Fuzzy Wuzzy Method

In [16]:
def similar_email_vold(user_email, user_emails):
    user_emails = Counter(user_emails)
    user_email_list = [i for i in user_emails]
    matches = process.extractBests(user_email, user_email_list, processor=match_process, scorer=match_score, score_cutoff=85, limit=20)
    scores = []
    sum_score = 0
    for match, confidence in matches:
        numTx = user_emails[match]
        if match != user_email:
            scores.append(numTx * 10)
    if scores:
        sum_score = min([sum(scores), 50])
        # if isEmoto(transaction):
        # this.addReason("\n<br>Extra emoto score", sum_score)
    return sum_score

In [17]:
%%time

user_emails = df['data.user_email'].tolist()[:1000]
user_emails = [email.lower() for email in list(set(user_emails))]

df4 = pd.DataFrame({'emails': list(user_emails)})
df4['score'] = 0
for idx, address in enumerate(tqdm(user_emails)):
    score = similar_email_vold(address, user_emails)
    df4.loc[idx, 'score'] = score


CPU times: user 1min 50s, sys: 893 ms, total: 1min 51s
Wall time: 1min 51s


In [46]:
df4.sort_values(by=['score'], ascending=0)

,emails,score
584,carenlou@gmail.com,20
495,tanya.andrew@gmail.com,20
551,chan_carisssa@yahoo.com,10
783,riyahkusniawati@gmail.com,10
517,carenlou0810@gmail.com,10
189,j-oyy@live.com,10
832,1114572634@qq.com,10
791,west2711@hotmail.com,10
793,www.riyahkusniawati@gmail.com,10
511,chan_carissa@yahoo.com,10


# Idea 3: Use Jellyfish Algorithms for Distances

In [90]:
import jellyfish

In [91]:
email_names = np.unique(df['data.user_email'].str.lower()).tolist()[:10000]

In [186]:
def get_jellyfish_similarity(email_names, name, top_n, algo='jellyfish.jaro_distance', reverse=True):
    """
    Compute similarity between two email addresses using the define distance measure and display top_n most similar
    email addresses together with its score
    
    email_names: list of all email_addresses to be scanned for similarity
    name: an email_add to be used as basis for searching most similar email address
    top_n: n most similar email addresses to display
    algo: jellyfish algorithm to be used for examining similarity
    reverse: identifier if reverse is used in sorting; this depends on the similarity measure used
    """
    
    top_n = sorted(list(map(lambda x: ((x, name),
            eval(algo)(x, name)), email_names)), key= lambda x: x[1], reverse=reverse)[:top_n]
    
    return list(filter(lambda x: x[1] > 0, top_n))

In [189]:
def get_similarity_matrix(email_names, top_n, algo, reverse):
    
    df = []
    
    for email in tqdm(email_names):
        sim_email = get_jellyfish_similarity(email_names, email, top_n, algo, reverse)
        sim_df = pd.DataFrame(sim_email, columns=['email_names','similarity_score'])
        sim_df[['email_1', 'email_2']] = sim_df['email_names'].apply(pd.Series)
        sim_df = sim_df.drop('email_names', axis=1)
        df.append(sim_df)
        
    return pd.concat(df, ignore_index=True)

### JARO DISTANCE

In [190]:
%%time

sim_jaro = get_similarity_matrix(email_names, 10, algo='jellyfish.jaro_distance', reverse=True)


CPU times: user 18min 34s, sys: 11.2 s, total: 18min 45s
Wall time: 19min 4s


In [193]:
sim_jaro = sim_jaro.sort_values(by=['similarity_score'], ascending=False)

In [195]:
sim_jaro[sim_jaro['email_1'].str.contains('carenlou@gmail.com')]

,similarity_score,email_1,email_2
91180,1.000000,carenlou@gmail.com,carenlou@gmail.com
91171,0.939394,carenlou@gmail.com,carenlou0810@gmail.com
90294,0.905093,carenlou@gmail.com,candolee@gmail.com
60263,0.901688,carenlou@gmail.com,auclove@gmail.com
72951,0.892266,carenlou@gmail.com,benlacount@gmail.com
28717,0.889498,carenlou@gmail.com,allencjho@gmail.com
83072,0.888889,carenlou@gmail.com,breloque@gmail.com
58524,0.888889,carenlou@gmail.com,asrielko@gmail.com
88171,0.888889,carenlou@gmail.com,caceyloh@gmail.com
93334,0.878307,carenlou@gmail.com,carolyuenaz@gmail.com


# Idea 4: Using Sequence Matcher

In [230]:
def get_sequencematcher_similarity(email_names, name, threshold, reverse=True):
    """
    Compute similarity between two email addresses using the define distance measure and display top_n most similar
    email addresses together with its score
    
    email_names: list of all email_addresses to be scanned for similarity
    name: an email_add to be used as basis for searching most similar email address
    top_n: n most similar email addresses to display
    algo: jellyfish algorithm to be used for examining similarity
    reverse: identifier if reverse is used in sorting; this depends on the similarity measure used
    """
    
    sort_sim = sorted(list(map(lambda x: ((x, name),SequenceMatcher(None,x,name).ratio()), email_names)), reverse=True)
    
    return list(filter(lambda x: x[1] > threshold, sort_sim))

In [231]:
def get_similarity_sequencematcher(email_names, threshold, reverse):
    
    df = []
    
    for email in tqdm(email_names):
        sim_email = get_sequencematcher_similarity(email_names, email, threshold, reverse)
        sim_df = pd.DataFrame(sim_email, columns=['email_names','similarity_score'])
        sim_df[['email_1', 'email_2']] = sim_df['email_names'].apply(pd.Series)
        sim_df = sim_df.drop('email_names', axis=1)
        df.append(sim_df)
        
    return pd.concat(df, ignore_index=True)

In [232]:
get_similarity_sequencematcher(email_names, 0.85, True)

,similarity_score,email_1,email_2
0,1.000000,000grace@hanmail.net,000grace@hanmail.net
1,1.000000,00116082@sats.com.sg,00116082@sats.com.sg
2,1.000000,00248024@sats.com.sg,00248024@sats.com.sg
3,1.000000,007banggu@naver.com,007banggu@naver.com
4,0.857143,77eileen@gmail.com,007elee@gmail.com
5,1.000000,007elee@gmail.com,007elee@gmail.com
6,1.000000,00wjddmswn00@hanmail.net,00wjddmswn00@hanmail.net
7,0.857143,4baobei@gmail.com,0104babi@gmail.com
8,1.000000,0104babi@gmail.com,0104babi@gmail.com
9,1.000000,0106vchng@gmail.com,0106vchng@gmail.com
